# <strong>Formação Cientista de Dados</strong>
## <font color=blue><strong>Criando um Modelo com Python, Usando `pyspark` com `LinearRegression`</strong></font>

### <font color=orange><strong>Carregando as bibliotecas importantes</strong></font>

* No código a seguir, está sendo realizado um processo de regressão utilizando a biblioteca **pyspark** em Python. Inicialmente, são importadas as classes necessárias, como **LinearRegression** e **RandomForestRegressor** para construir modelos de regressão. Em seguida, a **RegressionEvaluator** é utilizada para avaliar o desempenho dos modelos gerados. A biblioteca **VectorAssembler** é empregada para criar vetores de características a partir das colunas do conjunto de dados. A sessão do Spark é configurada através da criação de uma instância da classe **SparkSession**, com o nome "Regressão". Essa sessão é essencial para a execução de operações distribuídas. O código, em sua simplicidade, representa um ponto de partida para análises de regressão com o poder do Spark.

In [1]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession 

spark = SparkSession.builder.appName("Regressão").getOrCreate()

### <font color=orange><strong>Carregando e visualizando a base de dados no `Dataframe`</strong></font>

* No código a seguir, está sendo realizado o processo de leitura de dados de um arquivo CSV chamado "Carros.csv" utilizando o **Spark**. A variável **Carros_temp** recebe os dados, configurando a inferência do esquema automaticamente com o parâmetro **inferSchema** e considerando a existência de cabeçalho com **header=True**. Além disso, o separador de colunas está definido como ponto e vírgula através do parâmetro **sep=";"**. Em seguida, é utilizado o método **show()** para exibir uma prévia do DataFrame resultante. Esse código é útil para carregar e visualizar rapidamente dados de carros a partir de um arquivo CSV, facilitando análises posteriores no contexto de projetos relacionados à área de Tecnologia da Informação.

In [2]:
Carros_temp = spark.read.csv("Carros.csv", inferSchema = True, header = True, sep = ";")
Carros_temp.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
|    181|        6|        225|            276| 346| 2022|        1|          0|      3|          1|105|
|    143|        8|        360|            321| 357| 15

### <font color=orange><strong>Filtrando e visualizando o conjunto de dados de interesse</strong></font>

* No código a seguir, está sendo realizada uma operação de seleção de colunas em um DataFrame chamado Carros_temp. Através do método **select**, foram escolhidas as colunas "`Consumo`", "`Cilindros`", "`Cilindradas`" e "`HP`". Em seguida, a função **show()** é utilizada para exibir o resultado desse processo, apresentando as informações referentes às colunas selecionadas. Esse trecho de código é eficaz para filtrar e visualizar especificamente as variáveis de interesse no conjunto de dados, facilitando a análise e compreensão das informações contidas no DataFrame Carros_temp.

In [3]:
Carros = Carros_temp.select("Consumo", "Cilindros", "Cilindradas", "HP")
Carros.show()

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
|    181|        6|        225|105|
|    143|        8|        360|245|
|    244|        4|       1467| 62|
|    228|        4|       1408| 95|
|    192|        6|       1676|123|
|    178|        6|       1676|123|
|    164|        8|       2758|180|
|    173|        8|       2758|180|
|    152|        8|       2758|180|
|    104|        8|        472|205|
|    104|        8|        460|215|
|    147|        8|        440|230|
|    324|        4|        787| 66|
|    304|        4|        757| 52|
|    339|        4|        711| 65|
+-------+---------+-----------+---+
only showing top 20 rows



### <font color=orange><strong>Vetorizando e preparando as características da base de dados</strong></font>

* No código a seguir, está sendo realizado um processo de vetorização de atributos utilizando a biblioteca **VectorAssembler** em Python. Os atributos "`Consumo`", "`Cilindros`" e "`Cilindradas`" são especificados como entradas (**inputCols**), e a saída da vetorização é atribuída à variável **"caracteristicas"** (**outputCol**). Em seguida, a transformação é aplicada ao DataFrame **Carros** utilizando o método **transform**, resultando em uma nova coluna chamada **"caracteristicas"**. Por fim, o DataFrame modificado é exibido utilizando o método **show()**, proporcionando uma visualização dos dados transformados. Esse código é fundamental para preparar os dados, consolidando características relevantes em uma única coluna, facilitando análises subsequentes.

In [4]:
veccaracteristicas = VectorAssembler(inputCols = [("Consumo"), ("Cilindros"), ("Cilindradas")], outputCol = "caracteristicas")
Carros = veccaracteristicas.transform(Carros)
Carros.show()

+-------+---------+-----------+---+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|
+-------+---------+-----------+---+------------------+
|     21|        6|        160|110|  [21.0,6.0,160.0]|
|     21|        6|        160|110|  [21.0,6.0,160.0]|
|    228|        4|        108| 93| [228.0,4.0,108.0]|
|    214|        6|        258|110| [214.0,6.0,258.0]|
|    187|        8|        360|175| [187.0,8.0,360.0]|
|    181|        6|        225|105| [181.0,6.0,225.0]|
|    143|        8|        360|245| [143.0,8.0,360.0]|
|    244|        4|       1467| 62|[244.0,4.0,1467.0]|
|    228|        4|       1408| 95|[228.0,4.0,1408.0]|
|    192|        6|       1676|123|[192.0,6.0,1676.0]|
|    178|        6|       1676|123|[178.0,6.0,1676.0]|
|    164|        8|       2758|180|[164.0,8.0,2758.0]|
|    173|        8|       2758|180|[173.0,8.0,2758.0]|
|    152|        8|       2758|180|[152.0,8.0,2758.0]|
|    104|        8|        472|205| [104.0,8.0,472.0]|
|    104| 

### <font color=orange><strong>Realizando o particionando dos dados em conjuntos de treino e teste</strong></font>

* No código a seguir, está sendo realizado o particionamento dos dados em conjuntos de treino e teste para análise de carros. Através da função **randomSplit** da biblioteca **Carros**, os dados são divididos em proporções específicas, sendo 70% destinados ao conjunto de treino (CarrosTreino) e 30% ao conjunto de teste (CarrosTeste). Em seguida, são impressos os contadores de elementos em cada conjunto utilizando a função **count**. Esse procedimento é essencial para garantir uma abordagem eficaz de treinamento e avaliação de modelos, permitindo uma análise robusta das características dos carros.

In [5]:
CarrosTreino, CarrosTeste = Carros.randomSplit([0.7,0.3])
print(CarrosTreino.count())
print(CarrosTeste.count())

24
8


### <font color=orange><strong>Criando e treinando o modelo usando uma base de treino</strong></font>

* No código a seguir, está sendo utilizado o algoritmo de regressão linear em Python, utilizando a biblioteca **LinearRegression**. Aqui, a variável **reglin** representa a instância desse modelo, configurando os dados de entrada com as características sendo especificadas pela coluna "`caracteristicas`" e a variável alvo sendo representada pela coluna "`HP`". Posteriormente, o modelo é treinado com os dados de treino, representados pelo conjunto **CarrosTreino**, usando o método **fit**. Esse treinamento visa encontrar a relação linear entre as características fornecidas e a variável alvo, HP, permitindo previsões precisas com base nessa análise estatística. Essa abordagem simplificada é eficaz para análise preditiva em casos onde a relação entre variáveis é linear.

In [6]:
reglin = LinearRegression(featuresCol = "caracteristicas", labelCol = "HP")
modelo = reglin.fit(CarrosTreino)

### <font color=orange><strong>Realizando as previsões com o modelo previamente criado</strong></font>

* No código a seguir, está sendo realizada a previsão utilizando um modelo previamente treinado. A variável **previsao** armazena os resultados da transformação do conjunto de dados de teste (**CarrosTeste**) por meio da aplicação do modelo. A função **transform** é responsável por aplicar as transformações necessárias ao conjunto de teste. Posteriormente, os resultados são exibidos utilizando a função **show()**. Essa operação permite visualizar as previsões geradas pelo modelo para avaliar seu desempenho. Certifique-se de que o modelo em questão esteja devidamente treinado e que as bibliotecas relevantes estejam importadas para garantir a execução bem-sucedida do código.

In [7]:
previsao = modelo.transform(CarrosTeste)
previsao.show()

+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|    133|        8|        350|245| [133.0,8.0,350.0]|205.16189278715296|
|    158|        8|        351|264| [158.0,8.0,351.0]|202.92089145132786|
|    187|        8|        360|175| [187.0,8.0,360.0]| 200.2652016298153|
|    192|        6|       1676|123|[192.0,6.0,1676.0]|131.46556537597064|
|    192|        8|        400|175| [192.0,8.0,400.0]|199.53206447185596|
|    215|        4|       1201| 97|[215.0,4.0,1201.0]| 73.87972288329463|
|    228|        4|        108| 93| [228.0,4.0,108.0]| 80.54315065151368|
|    273|        4|         79| 66|  [273.0,4.0,79.0]| 76.72985217256776|
+-------+---------+-----------+---+------------------+------------------+



### <font color=orange><strong>Avaliando a peformance do modelo criado</strong></font>

* No código a seguir, está sendo realizado um processo de avaliação de desempenho em um modelo de regressão. A **avaliar** é uma instância da classe RegressionEvaluator, que recebe como parâmetros as colunas de predição ("`prediction`") e rótulos reais ("`HP`"), além da métrica escolhida, no caso, o erro quadrático médio raiz (RMSE). A variável **rmse** armazena o resultado da avaliação, calculado através do método evaluate aplicado na instância da avaliação, utilizando as previsões do modelo. O valor final do RMSE é então impresso, fornecendo uma métrica quantitativa da qualidade das predições, sendo uma medida comum para avaliação de modelos de regressão.

In [8]:
avaliar = RegressionEvaluator(predictionCol = "prediction", labelCol = "HP", metricName = 'rmse')
rmse = avaliar.evaluate(previsao)
rmse_percent = "{:.2%}".format(rmse/100)

print(f"Performance do modelo: {rmse_percent}")

Performance do modelo: 30.48%
